IMPORT LIBRARY

In [16]:
import requests, re, json, lxml
from bs4 import BeautifulSoup

PARAMETER PENCARIAN

In [17]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36",
  }

params = {
    "q": "bawang putih",              # search query
    "tbm": "isch",                # image results
    "hl": "id",                   # language of the search
    "gl": "id",                   # country where search comes fro
  }

SEARCH 

In [18]:
html = requests.get("https://google.com/search", params=params, headers=headers, timeout=30)
soup = BeautifulSoup(html.text, 'html.parser')
google_images = []
all_script_tags = soup.select("script")

REGEX (tinggal copas copas dari internet doang asli wkwk)

In [19]:
# https://regex101.com/r/48UZhY/4
matched_images_data = "".join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(all_script_tags)))

matched_images_data_fix = json.dumps(matched_images_data)
matched_images_data_json = json.loads(matched_images_data_fix)
  
# https://regex101.com/r/VPz7f2/1
matched_google_image_data = re.findall(r'\"b-GRID_STATE0\"(.*)sideChannel:\s?{}}', matched_images_data_json)

# https://regex101.com/r/Jt5BJW/1
matched_google_images_thumbnails = ", ".join(
    re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]',
                   str(matched_google_image_data))).split(", ")

thumbnails = [bytes(bytes(thumbnail, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for thumbnail in matched_google_images_thumbnails]

# removing previously matched thumbnails for easier full resolution image matches.
removed_matched_google_images_thumbnails = re.sub(
        r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', "", str(matched_google_image_data))

# https://regex101.com/r/fXjfb1/4
# https://stackoverflow.com/a/19821774/15164646
matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]", removed_matched_google_images_thumbnails)

full_res_images = [
        bytes(bytes(img, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for img in matched_google_full_resolution_images
]

METHOD DOWNLOAD

In [39]:
import requests
from PIL import Image
import io

def download(image,folder):
    # Set the image URL
    url = image

    # Make a GET request to the URL and save the response
    response = requests.get(url)

    # Open the response content as an image using Pillow
    images = Image.open(io.BytesIO(response.content))

    # Save the image to a file
    relname = ''.join(e for e in image if e.isalnum())
    images.save("{}/{}.jpg".format(folder,relname))

MASUKIN LINK KE ARRAY

In [21]:
for index, (metadata, thumbnail, original) in enumerate(zip(soup.select('.isv-r.PNCib.MSM1fd.BUooTd'), thumbnails, full_res_images), start=1):
    google_images.append(original)

In [45]:
google_images

['https://www.astronauts.id/blog/wp-content/uploads/2022/11/Manfaat-Bawang-Putih-Untuk-Kulit-yang-Tidak-Banyak-Tahu.jpg',
 'https://upload.wikimedia.org/wikipedia/commons/2/22/Garlic.jpg',
 'https://cdn-2.tstatic.net/belitung/foto/bank/images/20220716-manfaat-bawang-putih-12.jpg',
 'https://mysiloam-api.siloamhospitals.com/public-asset/website-cms/website-cms-16666826813597881.webp',
 'https://trubus.id/wp-content/uploads/2023/01/Manfaat-Bawang-Putih-sebagai-Obat-Luka-hingga-Pengawet-Tahu.jpg',
 'https://www.astronauts.id/blog/wp-content/uploads/2022/11/Kenali-Manfaat-Bawang-Putih-dan-Cara-Mengkonsumsinya-yang-Benar.jpg',
 'https://asset-a.grid.id/crop/0x0:0x0/x/photo/2020/12/21/3953776938.jpg',
 'https://storage.aido.id/articles/May2021/ieb0z3dbjhb2dyrk5h89.jpg',
 'https://images.tokopedia.net/img/cache/500-square/VqbcmM/2021/10/1/53612288-79c5-4957-ae41-ff1bccb12ef8.jpg',
 'https://res.cloudinary.com/dk0z4ums3/image/upload/v1617595463/attached_image/Apa-Manfaat-Bawang-Putih-bagi-Kese

DOWNLOAD

In [57]:
for i in google_images:
    download(i,folder='bawang_putih_code2_1link')

KeyboardInterrupt: 